In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model='Llama3-8b-8192',api_key=groq_api_key)
llm

In [ ]:
!pip install bs4

In [ ]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
import bs4
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content",'post-title','post-header')
        )
    ),
)
docs=loader.load()
docs

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

In [ ]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke(({"input":"What is Self reflection"}))
response

In [ ]:
response['answer']

In [ ]:
rag_chain.invoke(({"input":"how do we achieve it"}))

# Adding chat history

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt =(
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."

)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ('human','{input}'),
    ]
)

In [ ]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

In [ ]:
qa_prompt = ChatPromptTemplate(
    [
        ('system',system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []
question = "What is self reflection"
response1 = rag_chain.invoke({"input":question,'chat_history':chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1['answer'])
    ]
)

question2 = "How to achieve it?"
response2=rag_chain.invoke({'input':question,"chat_history":chat_history})
print(response2['answer'])

In [ ]:
chat_history

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer',
)

In [ ]:
conversational_rag_chain.invoke(
    {"input":"What is task decomposition?"},
    config={
        "configurable":{'session_id':"chat1"}

    },
)['answer']

In [ ]:
conversational_rag_chain.invoke({'input':"what are the commom ways of doing it?"},config={'configurable':{'session_id':'chat1'}},)['answer']

In [ ]:
store